---
title: 3.9 环境和分布偏移
date: 2024-3-17 14:00:00
tags: [机器学习,pytorch]
categories: [机器学习]
comment: true
toc: true
---
#
<!--more-->
# 9 环境和分布偏移
- （不好懂，建议跳过这一节）
- 在不同的应用场景模型决策方案会出现漏洞
## 9.1 分布偏移的类型
- 考虑一个二元分类问题：区分狗和猫。如果分布可以以任意方式偏移，那么我们的情景允许病态的情况，即输入的分布保持不变：$p_S(x) = p_T (x)$，但标签全部翻转：$p_S(y|x) = 1 − p_T (y|x)$。
### 9.1.1 协变量偏移
- 虽然输入的分布可能随时间而改变，但标签函数（即条件分布P(y | x)）没有改变。统计学家称之为协变量偏移（covariate shift），因为这个问题是由于协变量（特征）分布的变化而产生的。
- 比如用真的猫狗图像做训练，卡通猫狗做测试。
### 9.1.2 标签偏移
- 与协变量偏移相反。假设标签边缘概率P(y)可以改变，但是类别条件分布P(x | y)在不同的领域之间保持不变。当我们认为y导致x时，标签偏移是一个合理的假设。例如，预测患者的疾病，我们可能根据症状来判断，即使疾病的相对流行率随着时间的推移而变化。
- 在另一些情况下，标签偏移和协变量偏移假设可以同时成立。例如，当标签是确定的，即使y导致x，协变量偏移假设也会得到满足。有趣的是，在这些情况下，使用基于标签偏移假设的方法通常是有利的。这是因为这些方法倾向于包含看起来像标签（通常是低维）的对象，而不是像输入（通常是高维的）对象。
### 9.1.3 概念偏移
- 当标签的定义发生变化时，就会出现这种问题。
- 如果我们要建立一个机器翻译系统，P(y | x)的分布可能会因我们的位置不同而得到不同的翻译。这个问题可能很难被发现。所以，我们最好可以利用在时间或空间上逐渐发生偏移的知识。
## 9.2 分布偏移示例
### 9.2.1 医学诊断
- 假设我们想设计一个检测癌症的算法，从健康人和病人那里收集数据，然后训练算法。收集训练数据的分布和在实际中遇到的数据分布可能有很大的不同。用近乎完美的精度来区分健康和患病人群确实很容易。然而，这可能是因为受试者在年龄、激素水平、体力活动、饮食、饮酒以及其他许多与疾病无关的因素上存在差异。这对检测疾病的分类器可能并不适用。这些抽样可能会遇到极端的协变量偏移。此外，这种情况不太可能通过常规方法加以纠正。
### 9.2.2 自动驾驶汽车
- 对于一家想利用机器学习来开发自动驾驶汽车的公司，一个关键部件是“路沿检测器”。由于真实的注释数据获取成本很高，他们想出了一个“聪明”的想法：将游戏渲染引擎中的合成数据用作额外的训练数据。这对从渲染引擎中抽取的“测试数据”非常有效，但应用在一辆真正的汽车里真是一场灾难。正如事实证明的那样，路沿被渲染成一种非常简单的纹理。更重要的是，所有的路沿都被渲染成了相同的纹理，路沿检测器很快就学习到了这个“特征”。
- 当美军第一次试图在森林中探测坦克时，也发生了类似的事情。他们在没有坦克的情况下拍摄了森林的航拍照片，然后把坦克开进森林，拍摄了另一组照片。使用这两组数据训练的分类器似乎工作得很好。不幸的是，分类器仅仅学会了如何区分有阴影的树和没有阴影的树：第一组照片是在清晨拍摄的，而第二组是在中午拍摄的。
### 9.2.3 非平稳分布
- 当分布变化缓慢并且模型没有得到充分更新时，就会出现更微妙的情况：非平稳分布（nonstationary distri‐bution）
    - 训练一个计算广告模型，但却没有经常更新（例如，一个2009年训练的模型不知道一个叫iPad的不知名新设备刚刚上市）；
    - 建立一个垃圾邮件过滤器，它能很好地检测到所有垃圾邮件。但是，垃圾邮件发送者们变得聪明起来，制造出新的信息，看起来不像我们以前见过的任何垃圾邮件；
    - 建立一个产品推荐系统，它在整个冬天都有效，但圣诞节过后很久还会继续推荐圣诞帽。

## 9.3 分布偏移纠正
- 通过一些策略应对分布偏移
### 9.3.1 经验风险与实际风险
- 不考虑正则化，训练损失为：$\operatorname * {minimize}_ f \frac1n \sum_ {i=1}^ nl(f( \mathbf{x}_ i),y_ i)$，也称经验风险，其中$l$是损失函数。
- 经验风险是为了近似真实风险：整个训练数据上的平均损失，即从真实分布$p(x,y)$中抽取的所有数据的总体损失的期望值：
    $$E_{p(\mathbf{x},y)}[l(f(\mathbf{x}),y)]=\int\int l(f(\mathbf{x}),y)p(\mathbf{x},y)~d\mathbf{x}dy$$
    然而，我们通常无法获得总体数据。

### 9.3.2 协变量偏移纠正
- 假设对于带标签的数据(xi, yi)，我们要评估P(y | x)。然而观测值xi是从某些源分布q(x)中得出的，而不是从目标分布p(x)中得出的。幸运的是，依赖性假设意味着条件分布保持不变，即：p(y | x) = q(y | x)。如果源分布q(x)是“错误的”，我们可以通过在真实风险的计算中，使用以下简单的恒等式来进行纠正：
$$\begin{aligned}\int\int l(f(\mathbf{x}),y)p(y\mid\mathbf{x})p(\mathbf{x})d\mathbf{x}dy&=\int\int l(f(\mathbf{x}),y)q(y\mid\mathbf{x})q(\mathbf{x})\frac{p(\mathbf{x})}{q(\mathbf{x})}d\mathbf{x}dy\end{aligned}$$
- 因此我们需要根据数据来自正确分布与来自错误分布的概率之比，来重新衡量每个数据样本的权重：
$$\beta_i\stackrel{\mathrm{def}}{=}\frac{p(\mathbf{x}_i)}{q(\mathbf{x}_i)}$$
- 将权重带入每个数据样本，得到目标函数：加权经验风险最小化
$$\min_f\text{mize }\frac1n\sum_{i=1}^n\beta_il(f(\mathbf{x}_i),y_i)$$
- 由于不知道这个比率，我们需要估计它。需要从两个分布中抽取样本：“真实”的分布p，通过访问测试数据获取；训练集q，通过人工合成的很容易获得。我们只需要特征x ∼ p(x)，不需要访问标签y ∼ p(y)。
- 一种非常有效的方法：对数几率回归，这是用于二元分类的softmax回归的一个特例。我们学习了一个分类器来区分从p(x)抽取的数据和从q(x)抽取的数据。如果无法区分这两个分布，则意味着相关的样本可能来自这两个分布中的任何一个。另一方面，任何可以很好区分的样本都应该相应地显著增加或减少权重。
- 假设我们分别从p(x)和q(x) 两个分布中抽取相同数量的样本。现在用z标签表示：从p抽取的数据为1，从q抽取的数据为−1。然后，混合数据集中的概率由下式给出
$$P(z=1\mid\mathbf{x})=\frac{p(\mathbf{x})}{p(\mathbf{x})+q(\mathbf{x})}\text{ and hence }\frac{P(z=1\mid\mathbf{x})}{P(z=-1\mid\mathbf{x})}=\frac{p(\mathbf{x})}{q(\mathbf{x})}$$
- 因此，如果我们使用对数几率回归方法，其中$P(z=1\mid\mathbf{x})=\frac{1}{1+\exp(-h(\mathbf{x}))}$（h是一个参数化函数），则（？？？）
$$\beta_i=\frac{1/(1+\exp(-h(\mathbf{x}_i)))}{\exp(-h(\mathbf{x}_i))/(1+\exp(-h(\mathbf{x}_i)))}=\exp(h(\mathbf{x}_i))$$
- 因此，我们需要解决两个问题：第一个问题是关于区分来自两个分布的数据；第二个问题是关于 (4.9.5) 中的
加权经验风险的最小化问题。
- 假设我们有一个训练集{$(x_1,y_1),···，(x_n,y_n)$}和一个未标记的测试集{$u_1,···,u_m$}.假设1 $\leq$ i $\leq$ n的$x_i$来自某个源分布，$u_i$来自目标分布。纠正协变量偏移的典型算法：
    1. 生成一个二元分类训练集：{$(x_1,-1),···,(x_n,-1),(u_1,1),···,(u_m,1)$}
    2. 用对数几率回归训练二元分类器得到函数h。
    3. 使用$\beta_i = exp(h(x_i))$或更好的$\beta_i = min(exp(h(x_i)),c)$(c为常量)对训练数据进行加权。
    4. 使用权重$\beta_i$进行加权经验风险最小化的训练。
- 上述算法依赖于一个重要的假设：需要目标分布(例如，测试分布)中的每个数据样本在训练时出现的概率非零。如果我们找到p(x) > 0但q(x) = 0的点，那么相应的重要性权重会是无穷大。
### 9.3.3 标签偏移纠正
- 假设标签的分布随时间变化：q(y) ̸= p(y)，但类别条件分布保持不变：q(x | y) = p(x | y)。如果源分布q(y)是“错误的”，我们可以根据真实风险的定义进行更正：
$$\int\int l(f(\mathbf{x}),y)p(\mathbf{x}\mid y)p(y)d\mathbf{x}dy=\int\int l(f(\mathbf{x}),y)q(\mathbf{x}\mid y)q(y)\frac{p(y)}{q(y)}d\mathbf{x}dy$$
- 重要性权重应用于标签似然比率：
$$\beta_i\stackrel{\mathrm{def}}{=}\frac{p(y_i)}{q(y_i)}$$
- 为了估计目标标签分布，我们首先采用性能相当好的现成的分类器（通常基于训练数据进行训练），并使用验证集（也来自训练分布）计算其混淆矩阵。混淆矩阵C是一个k × k矩阵，其中每列对应于标签类别，每行对应于模型的预测类别。每个单元格的值cij是验证集中，真实标签为j，而我们的模型预测为i的样本数量所占的比例。
- 我们不能直接计算目标数据上的混淆矩阵，因为我们无法看到真实环境下的样本的标签，除非我们再搭建一个复杂的实时标注流程。然而，我们所能做的是将所有模型在测试时的预测取平均数，得到平均模型输出$\mu(\hat{\mathbf{y}})\in\mathbb{R}^k$，其中第i个元素$\mu(\hat y_i)$是我们模型预测测试集中i的总预测分数。
- 如果我们的分类器一开始就相当准确，并且目标数据只包含我们以前见过的类别，以及如果标签偏移假设成立（这里最强的假设），我们就可以通过求解一个简单的线性系统来估计测试集的标签分布
$$\mathbf{C}p(\mathbf{y})=\mu(\hat{\mathbf{y}})$$
- 因为作为一个估计，$\begin{aligned}\sum_{j=1}^kc_{ij}p(y_j)=\mu(\hat{y}_i)\end{aligned}$对所有$1\leq i \leq k$成立，其中$p(y_i)$是k维标签分布向量p(y)的第$j^{th}$元素。如果我们的分类器一开始就足够精确，那么混淆矩阵$\mathbf C$将是可逆的，进而我们可以得到一个解$p(y)=\mathbf C^{-1}\mu(\hat y)$因为我们观测源数据上的标签，所以很容易估计分布q(y)。那么对于标签为yi的任何训练样本i，我们可以使用我们估计的p(yi)/q(yi)比率来计算权重$\beta_i$，并将其代入加权经验风险最小化中。
### 9.3.4 概念偏移纠正
- 概念偏移很难用原则性的方式解决。例如，在一个问题突然从“区分猫和狗”偏移为“区分白色和黑色动物”的情况下，除了从零开始收集新标签和训练，别无妙方。幸运的是，在实践中这种极端的偏移是罕见的。相反，通常情况下，概念的变化总是缓慢的。比如下面是一些例子：
    - 在计算广告中，新产品推出后，旧产品变得不那么受欢迎了。这意味着广告的分布和受欢迎程度是逐渐
    变化的，任何点击率预测器都需要随之逐渐变化；
    - 由于环境的磨损，交通摄像头的镜头会逐渐退化，影响摄像头的图像质量；
    - 新闻内容逐渐变化（即新新闻的出现）。
- 在这种情况下，我们可以使用与训练网络相同的方法，使其适应数据的变化。换言之，我们使用新数据更新现有的网络权重，而不是从头开始训练。
## 9.4 学习问题的分类法

### 9.4.1 批量学习
- 在批量学习（batch learning）中，我们可以访问一组训练特征和标签 {(x1, y1), . . . ,(xn, yn)}，我们使用这些特性和标签训练f(x)。然后，我们部署此模型来对来自同一分布的新数据(x, y)进行评分。例如，我们可以根据猫和狗的大量图片训练猫检测器。一旦我们训练了它，我们就把它作为智能猫门计算视觉系统的一部分，来控制只允许猫进入。然后这个系统会被安装在客户家中，基本再也不会更新。

### 9.4.2 在线学习
- 除了“批量”地学习，我们还可以单个“在线”学习数据(xi, yi)。更具体地说，我们首先观测到xi，然后我们得出一个估计值f(xi)，只有当我们做到这一点后，我们才观测到yi。然后根据我们的决定，我们会得到奖励或损失。许多实际问题都属于这一类。例如，我们需要预测明天的股票价格，这样我们就可以根据这个预测进行交易。在一天结束时，我们会评估我们的预测是否盈利。换句话说，在在线学习（online learning）中，我们有以下的循环。在这个循环中，给定新的观测结果，我们会不断地改进我们的模型。
$$ \text{model }f_ t \longrightarrow \text{data } \mathbf{x}_ t \longrightarrow \text{estimate }f_ t( \mathbf{x}_ t) \longrightarrow \text{observation }y_t \longrightarrow \text{loss }l(y_ t,f_ t( \mathbf{x}_ t)) \longrightarrow \text{model }f_ {t+1}$$

### 9.4.3 老虎机
- 老虎机（bandits）是上述问题的一个特例。虽然在大多数学习问题中，我们有一个连续参数化的函数f（例如，一个深度网络）。但在一个老虎机问题中，我们只有有限数量的手臂可以拉动。也就是说，我们可以采取的行动是有限的。对于这个更简单的问题，可以获得更强的最优性理论保证，这并不令人惊讶。我们之所以列出它，主要是因为这个问题经常被视为一个单独的学习问题的情景。
### 9.4.4 控制
- 在很多情况下，环境会记住我们所做的事。不一定是以一种对抗的方式，但它会记住，而且它的反应将取决于之前发生的事情。例如，咖啡锅炉控制器将根据之前是否加热锅炉来观测到不同的温度。在这种情况下，PID（比例—积分—微分）控制器算法是一个流行的选择。同样，一个用户在新闻网站上的行为将取决于之前向她展示的内容（例如，大多数新闻她只阅读一次）。许多这样的算法形成了一个环境模型，在这个模型中，他们的行为使得他们的决策看起来不那么随机。近年来，控制理论（如PID的变体）也被用于自动调整超参数，以
获得更好的解构和重建质量，提高生成文本的多样性和生成图像的重建质量。
### 9.4.5 强化学习
- 强化学习（reinforcement learning）强调如何基于环境而行动，以取得最大化的预期利益。国际象棋、围棋、西洋双陆棋或星际争霸都是强化学习的应用实例。再比如，为自动驾驶汽车制造一个控制器，或者以其他方式对自动驾驶汽车的驾驶方式做出反应（例如，试图避开某物体，试图造成事故，或者试图与其合作）。
### 9.4.6 考虑到环境
- 上述不同情况之间的一个关键区别是：在静止环境中可能一直有效的相同策略，在环境能够改变的情况下可能不会始终有效。例如，一个交易者发现的套利机会很可能在他开始利用它时就消失了。环境变化的速度和方式在很大程度上决定了我们可以采用的算法类型。例如，如果我们知道事情只会缓慢地变化，就可以迫使任何估计也只能缓慢地发生改变。如果我们知道环境可能会瞬间发生变化，但这种变化非常罕见，我们就可以在使用算法时考虑到这一点。当一个数据科学家试图解决的问题会随着时间的推移而发生变化时，这些类
型的知识至关重要。
